In [9]:
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
sys.path.append(str(src_path))

import json
import types
from collections import OrderedDict
from pprint import pprint

import torch
import torch.nn as nn
from dataset import get_new_dataloader
from funcs import get_dataloaders
from models import PreActResNet18, WideResNet
from train import epoch, epoch_test


In [10]:
# constant
CWD_PATH = Path.cwd()
MODEL_DIR_PATH = CWD_PATH.parent / "models/CIFAR10-diff-norm-value-dist"
SETTINGS = [
    {
        "normalization_mean": [0.547, 0.539, 0.507],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.603, 0.596, 0.567],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.659, 0.652, 0.628],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.715, 0.709, 0.688],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.771, 0.766, 0.748],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.799, 0.794, 0.779],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.827, 0.823, 0.809],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.855, 0.851, 0.839],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.883, 0.879, 0.869],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.911, 0.908, 0.899],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.939, 0.936, 0.93],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.967, 0.965, 0.96],
        "normalization_std": None,
    },
    {
        "normalization_mean": [0.995, 0.993, 0.99],
        "normalization_std": None,
    },
]


In [11]:
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    print(label)

model__mean_0.547_0.539_0.507
model__mean_0.603_0.596_0.567
model__mean_0.659_0.652_0.628
model__mean_0.715_0.709_0.688
model__mean_0.771_0.766_0.748
model__mean_0.799_0.794_0.779
model__mean_0.827_0.823_0.809
model__mean_0.855_0.851_0.839
model__mean_0.883_0.879_0.869
model__mean_0.911_0.908_0.899
model__mean_0.939_0.936_0.93
model__mean_0.967_0.965_0.96
model__mean_0.995_0.993_0.99


In [12]:
def load_model(
    model_type: str,
    location: str,
    model_normalize: bool,
):
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")

    # load model CIFAR10
    if model_type == "pre-act-18":
        model = PreActResNet18(num_classes=10, normalize=model_normalize)
    elif model_type == "independent":
        model = WideResNet(
            n_classes=10,
            depth=16,  # deep_full for CIFAR10
            widen_factor=1,
            normalize=model_normalize,
            dropRate=0.3,
        )
    else:
        model = WideResNet(
            n_classes=10,
            depth=28,  # deep_full for CIFAR10
            widen_factor=10,
            normalize=model_normalize,
            dropRate=0.3,
        )

    try:
        model = model.to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    except:
        model = nn.DataParallel(model).to(device)
        model.load_state_dict(torch.load(location, map_location=device))
    model.eval()
    return model


In [13]:
model_name__tr_acc = OrderedDict()

# model performance in TR
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"
    args.experiment = 'diff-norm-value'
    args.dataset = 'CIFAR10'
    args.batch_size = 1000
    args.num_workers = 8
    args.normalization_mean = normalization_mean
    args.normalization_std = normalization_std
    args.extra_preprocessing_type = ""
    train_loader, test_loader = get_new_dataloader(args=args)

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__tr_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }


/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.547_0.539_0.507/final.pt
Custom_normalization: Normalize(mean=[0.547, 0.539, 0.507], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:28<00:00,  1.76it/s]


1.0
0.9422
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.603_0.596_0.567/final.pt
Custom_normalization: Normalize(mean=[0.603, 0.596, 0.567], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


0.99998
0.9427
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.659_0.652_0.628/final.pt
Custom_normalization: Normalize(mean=[0.659, 0.652, 0.628], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


...

0.99998
0.9421
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.967_0.965_0.96/final.pt
Custom_normalization: Normalize(mean=[0.967, 0.965, 0.96], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


0.99998
0.9416
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.995_0.993_0.99/final.pt
Custom_normalization: Normalize(mean=[0.995, 0.993, 0.99], std=(0.2471, 0.2435, 0.2616))
Lambda()
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:23<00:00,  2.15it/s]


0.99996
0.9462


In [14]:
pprint(model_name__tr_acc)

# Serializing json
json_object = json.dumps(model_name__tr_acc, indent=2)
 
# Writing to sample.json
with open("tr_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_0.547_0.539_0.507',
              {'test_acc': 0.9422, 'train_acc': 1.0}),
             ('model__mean_0.603_0.596_0.567',
              {'test_acc': 0.9427, 'train_acc': 0.99998}),
             ('model__mean_0.659_0.652_0.628',
              {'test_acc': 0.9401, 'train_acc': 0.99996}),
             ('model__mean_0.715_0.709_0.688',
              {'test_acc': 0.9419, 'train_acc': 0.99994}),
             ('model__mean_0.771_0.766_0.748',
              {'test_acc': 0.9456, 'train_acc': 0.99994}),
             ('model__mean_0.799_0.794_0.779',
              {'test_acc': 0.941, 'train_acc': 0.99998}),
             ('model__mean_0.827_0.823_0.809',
              {'test_acc': 0.9404, 'train_acc': 1.0}),
             ('model__mean_0.855_0.851_0.839',
              {'test_acc': 0.9438, 'train_acc': 1.0}),
             ('model__mean_0.883_0.879_0.869',
              {'test_acc': 0.9422, 'train_acc': 1.0}),
             ('model__mean_0.911_0.908_0.899',
              {'

In [15]:
model_name__te_acc = OrderedDict()

# model performance in FE
for setting in SETTINGS:
    normalization_mean = setting['normalization_mean']
    normalization_std = setting['normalization_std']
    if normalization_mean != None and normalization_std != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}_std_{std_str}"
    elif normalization_mean != None:
        mean_str = str(normalization_mean).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__mean_{mean_str}"
    elif normalization_std != None:
        std_str = str(normalization_std).replace(' ', '').replace(',', '_')[1:-1]
        label = f"model__std_{std_str}"
    else:
        label = f"model__baseline"

    model_path = MODEL_DIR_PATH / label / "final.pt"
    print(model_path)
    model = load_model(
        model_type="teacher",
        location=str(model_path),
        model_normalize=False,
    )

    train_loader, test_loader = get_dataloaders(
        dataset="CIFAR10",
        batch_size=1000,
        pseudo_labels=False,
        normalize=True,
        train_shuffle=False,
    )

    args = types.SimpleNamespace()
    device = torch.device("cuda:{0}".format(0) if torch.cuda.is_available() else "cpu")
    args.device = device
    args.mode = "teacher"

    _, train_acc = epoch(
        args,
        train_loader,
        model,
        teacher=None,
        lr_schedule=None,
        epoch_i=None,
        opt=None,
    )
    _, test_acc = epoch_test(
        args,
        test_loader,
        model,
    )
    print(train_acc)
    print(test_acc)

    model_name__te_acc[label] = {
        "train_acc": train_acc,
        "test_acc": test_acc,
    }



/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.547_0.539_0.507/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


0.99992
0.9386
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.603_0.596_0.567/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.99978
0.9359
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.659_0.652_0.628/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


...

0.96128
0.884
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.967_0.965_0.96/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.94888
0.875
/data/weijing/DI-ting_normalization/models/CIFAR10-diff-norm-value-dist/model__mean_0.995_0.993_0.99/final.pt
No Transform
Files already downloaded and verified
Files already downloaded and verified
No Transform
Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 50/50 [00:31<00:00,  1.59it/s]


0.94516
0.8755


In [16]:
pprint(model_name__te_acc)

# Serializing json
json_object = json.dumps(model_name__te_acc, indent=2)
 
# Writing to sample.json
with open("fe_performance.json", "w") as outfile:
    outfile.write(json_object)

OrderedDict([('model__mean_0.547_0.539_0.507',
              {'test_acc': 0.9386, 'train_acc': 0.99992}),
             ('model__mean_0.603_0.596_0.567',
              {'test_acc': 0.9359, 'train_acc': 0.99978}),
             ('model__mean_0.659_0.652_0.628',
              {'test_acc': 0.9351, 'train_acc': 0.99942}),
             ('model__mean_0.715_0.709_0.688',
              {'test_acc': 0.9282, 'train_acc': 0.99784}),
             ('model__mean_0.771_0.766_0.748',
              {'test_acc': 0.9177, 'train_acc': 0.99378}),
             ('model__mean_0.799_0.794_0.779',
              {'test_acc': 0.9122, 'train_acc': 0.98778}),
             ('model__mean_0.827_0.823_0.809',
              {'test_acc': 0.9058, 'train_acc': 0.98672}),
             ('model__mean_0.855_0.851_0.839',
              {'test_acc': 0.9065, 'train_acc': 0.983}),
             ('model__mean_0.883_0.879_0.869',
              {'test_acc': 0.8975, 'train_acc': 0.974}),
             ('model__mean_0.911_0.908_0.899',
   